In [1]:
import sys
sys.path.append('..')
from mtDNAsim import *
import pandas as pd
import numpy as np
from copy import deepcopy
from collections import Counter

/home/wangkun/mtDNA_simulation/notebook/../mtDNAsim/sim_utils.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
filename = 'test'

In [4]:
num_elements = 5
system = Gillespie(
    num_elements,
    inits=[1, 0, 0, 0, 0],
    max_cell_num=8000
)

p0 = lambda t: (1 - 1 / (1 + np.exp(-0.6 * (t - 17))))
p1 = lambda t: (1 - 1 / (1 + np.exp(-0.6 * (t - 17))))
p2 = lambda t: (1 - 1 / (1 + np.exp(-0.6 * (t - 17))))
p3 = lambda t: (1 - 1 / (1 + np.exp(-0.6 * (t - 18))))
p4 = lambda t: (1 - 1 / (1 + np.exp(-0.6 * (t - 19))))
d0 = lambda t: 1 - p0(t)
d1 = lambda t: 1 - p1(t)
d2 = lambda t: 1 - p2(t)
d3 = lambda t: 1 - p3(t)

system.add_reaction(p0, [1, 0, 0, 0, 0], [2, 0, 0, 0, 0], index=0) # 0 self renew
system.add_reaction(p1, [0, 1, 0, 0, 0], [0, 2, 0, 0, 0], index=1) # 1 self renew
system.add_reaction(p2, [0, 0, 1, 0, 0], [0, 0, 2, 0, 0], index=2) # 2 self renew
system.add_reaction(p3, [0, 0, 0, 1, 0], [0, 0, 0, 2, 0], index=3) # 3 self renew
system.add_reaction(p4, [0, 0, 0, 0, 1], [0, 0, 0, 0, 2], index=4) # 4 self renew
system.add_reaction(d0, [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], index=5) # 0 -> 1 differentiation
system.add_reaction(d1, [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], index=6) # 1 -> 2 differentiation
system.add_reaction(d2, [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], index=7) # 2 -> 3 differentiation
system.add_reaction(d3, [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], index=8) # 3 -> 4 differentiation

system.evolute(20000000)

data_path = "./"
tree_file_name = f"tree_origin_linear_{filename}.csv"
cell_num_file_name = f"cell_num_linear_{filename}.csv"

curr_cells = []
t = np.array(system.generation_time)
cell_num_traj = np.array(system.n)

for i in system.curr_cells.values():
    curr_cells += i
# while tree_file_name in os.listdir(data_path):
#     tree_file_name = tree_file_name[:-1] + str(int(tree_file_name[-1]) + 1)
#     cell_num_file_name = cell_num_file_name[:-1] + str(int(cell_num_file_name[-1]) + 1)

np.savetxt(
    data_path + cell_num_file_name,
    np.hstack((t.reshape(-1, 1), cell_num_traj)),
    fmt="%.5f",
)

sim_utils.wirte_lineage_info(
    data_path + tree_file_name, system.anc_cells, curr_cells, system.t[-1]
)
try:
    reconstruct(f'./tree_origin_linear_{filename}.csv', output=f'./linear_tree_gt_{filename}.nwk', num=100, is_balance=True)
except:
    os.system(f'rm ../results/{tree_file_name}')
    os.system(f'rm ../results/{cell_num_file_name}')

tree_file = f'./linear_tree_gt_{filename}.nwk'
phylo_tree, branch_colors = loadtree(tree_file)
sampled_cells = [i.name for i in phylo_tree.get_terminals()]
# cell_names, cell_states, cell_generation = get_annotation('../results/tree_origin_linear.csv')
# cell_states = pd.DataFrame(data=cell_states, index=cell_names).loc[sampled_cells]
# cell_generation = pd.DataFrame(data=cell_generation, index=cell_names).loc[sampled_cells].to_numpy()

  0%|          | 0/8000 [00:00<?, ?it/s]

In [5]:
seqs = DNAmutation(phylo_tree, mut_rate=1.5)
seqs = seqs.astype(int)

success = 0
while not success:
    try:
        mt_muts, mutid = mtmutation(phylo_tree, mut_rate=0.0016, init_mut_rate=5, mt_copynumber=2, nmts=500)
        success = 1
    except:
        None

pre_existing_mut = set()
for i in mt_muts['<0_0>']:
    pre_existing_mut = pre_existing_mut.union(i)

mf = mut_freq(mt_muts, mutid, sel_cells=[i.name for i in phylo_tree.get_terminals()])

mt_seqs = mf.astype(bool).astype(int)
mt_pre = mt_seqs[mt_seqs.columns[np.isin(mt_seqs.columns, list(pre_existing_mut))]]
mt_dn =  mt_seqs[mt_seqs.columns[~np.isin(mt_seqs.columns, list(pre_existing_mut))]]

  0%|          | 0/499 [00:00<?, ?it/s]

  0%|          | 0/1072 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
mf[mf<0.05] = 0

In [17]:
mf

,0,1,2,3,4,5,6,7,8,9,...,6981,6989,6990,6991,6992,6997,7003,7004,7008,7016
<23_878>,0.252727,0.252727,0.252727,0.252727,0.252727,0.747273,0.747273,0.747273,0.747273,0.747273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<20_1802>,0.274319,0.274319,0.274319,0.274319,0.274319,0.725681,0.725681,0.725681,0.725681,0.725681,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<25_531>,0.305369,0.305369,0.305369,0.305369,0.305369,0.694631,0.694631,0.694631,0.694631,0.694631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<22_1351>,0.225869,0.225869,0.225869,0.225869,0.225869,0.774131,0.774131,0.774131,0.774131,0.774131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<18_2019>,0.244048,0.244048,0.244048,0.244048,0.244048,0.755952,0.755952,0.755952,0.755952,0.755952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
<14_1452>,0.392344,0.392344,0.392344,0.392344,0.392344,0.607656,0.607656,0.607656,0.607656,0.607656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<14_1109>,0.223062,0.223062,0.223062,0.223062,0.223062,0.776938,0.776938,0.776938,0.776938,0.776938,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<13_1212>,0.335185,0.335185,0.335185,0.335185,0.335185,0.664815,0.664815,0.664815,0.664815,0.664815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<13_979>,0.323308,0.323308,0.323308,0.323308,0.323308,0.676692,0.676692,0.676692,0.676692,0.676692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
with open(f'../results/mt_mut_{filename}.phy', 'w') as f:
    f.write('{} {}\n'.format(*mt_seqs.shape))
    for cell in mt_seqs.index:
        f.write('{} {}\n'.format(cell[1:-1], ''.join(mt_seqs.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))

with open(f'../results/mt_mut_pre_{filename}.phy', 'w') as f:
    f.write('{} {}\n'.format(*mt_pre.shape))
    for cell in mt_pre.index:
        f.write('{} {}\n'.format(cell[1:-1], ''.join(mt_pre.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))
        
with open(f'../results/mt_mut_dn_{filename}.phy', 'w') as f:
    f.write('{} {}\n'.format(*mt_dn.shape))
    for cell in mt_dn.index:
        f.write('{} {}\n'.format(cell[1:-1], ''.join(mt_dn.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))        
        

with open(f'../results/dna_mut_{filename}.phy', 'w') as f:
    f.write('{} {}\n'.format(*seqs.shape))
    for cell in seqs.index:
        f.write('{} {}\n'.format(cell[1:-1], ''.join(seqs.loc[cell].astype(str)).replace('0', 'A').replace('1', 'G')))
        